# Handle Maven dependency issues

Explore using the task runner, to handle both maven compiler errors when dependencies are missing, and a specific task from the analyzer-lsp validator

## Setting up the environment

In [1]:
import pprint

pp = pprint.PrettyPrinter(indent=2)

## Create the Test Environment

For creating the test environment we will:

1. move the coolstore to a temp direcotry
2. Update the InventoryEntity to use UUID for the itemID as the type.

In [2]:
import tempfile
import shutil
import os
import subprocess

#First we need to download coolstore if it does not exist.

if not os.path.exists("./coolstore"):
    subprocess.run("../../example/fetch.sh")

temp_dir = tempfile.TemporaryDirectory()
coolstore_path = os.path.join(temp_dir.name, "coolstore")
shutil.copytree("./coolstore", coolstore_path)

## make the necessary change
shutil.copyfile("./testing_pom_changes/pom.xml", f"{coolstore_path}/pom.xml")
print(temp_dir)


<TemporaryDirectory '/var/folders/vt/5bfp7vyd1h79_7k5ygr0fttr0000gn/T/tmpqdcjrh3t'>


## Testing for Dependency issue with coolstore missing a dependencies

This will test that removed dependency is re-added 

In [3]:
from pathlib import Path
from kai.service.llm_interfacing.model_provider import ModelProvider
from kai.models.kai_config import KaiConfig
from playpen.repo_level_awareness.agent.dependency_agent import MavenDependencyAgent
from playpen.repo_level_awareness.api import RpcClientConfig
from playpen.repo_level_awareness.codeplan import TaskManager
from playpen.repo_level_awareness.task_runner.compiler.maven_validator import MavenCompileStep, SymbolNotFoundError, PackageDoesNotExistError
from playpen.repo_level_awareness.task_runner.dependency.task_runner import DependencyTaskRunner
import os

from playpen.repo_level_awareness.vfs.git_vfs import RepoContextManager

config = RpcClientConfig(Path(coolstore_path), None, None, None, None, None)
kai_config = KaiConfig.model_validate_filepath("../../kai/config.toml")
modelProvider = ModelProvider(kai_config.models)
rcm = RepoContextManager(config.repo_directory, modelProvider.llm)
agent = MavenDependencyAgent(modelProvider.llm)

# Create the task runner.
task_runner = DependencyTaskRunner(agent)

## Now, we need to set up code plan, with this agent, and to talk to coolstore. To show the missing dependencies we will only use the maven compiler validator.

task_manager = TaskManager(
        config,
        rcm,
        None,
        validators=[MavenCompileStep(config)],
        agents=[task_runner],
    )


for task in task_manager.get_next_task():
    pp.pprint(task)
    if isinstance(task, PackageDoesNotExistError):
        task_manager.supply_result(task_manager.execute_task(task))


100
PackageDoesNotExistError(file='/private/var/folders/vt/5bfp7vyd1h79_7k5ygr0fttr0000gn/T/tmpqdcjrh3t/coolstore/src/main/java/com/redhat/coolstore/utils/DataBaseMigrationStartup.java',
                         line=6,
                         column=24,
                         message='package javax.annotation does not exist',
                         details=[],
                         parse_lines='[ERROR] '
                                     '/private/var/folders/vt/5bfp7vyd1h79_7k5ygr0fttr0000gn/T/tmpqdcjrh3t/coolstore/src/main/java/com/redhat/coolstore/utils/DataBaseMigrationStartup.java:[6,24] '
                                     'package javax.annotation does not exist',
                         missing_package='javax.annotation')
_llm_response(actions=[_action(code='result = search_fqdn.run(artifact_id="javax.annotation", group_id="javax.annotation")', thought=' The error message indicates that the Maven compiler is unable to find the javax.annotation package, which sugg

AttributeError: 'dict' object has no attribute 'response'

## Cleanup temp dir

In [4]:
temp_dir.cleanup()